In [1]:
import spotipy
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)


# client_id = '12e4e34e23e841b3aa386f3e1e9afa74'
# client_secret = '1e5335d90d44414495573690e004d3ae'
client_id = 'dab809ffa57e4712981d068a2dbcc8f6'
client_secret = 'd0e73dc83d4d45e7aaea970f95de6c23'

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [2]:
df = pd.read_csv('./dataset/unique_min_rank.csv', header=0, encoding = "ISO-8859-1")
df = df.drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], axis=1)
df.head()

/tmp/ipykernel_2123/2507618338.py:1: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/unique_min_rank.csv', header=0, encoding = "ISO-8859-1")


,title,rank,date,artist,url,region,chart,trend,streams,min_rank
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153


In [3]:
print(df.shape)
df = df.dropna()
df.shape

(299532, 10)


(296597, 10)

In [4]:
from tqdm import tqdm

tqdm.pandas()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
# concatenate all features to dataframe row by row
def get_all_features(df_row):
    global row_cnt, failed_cnt
    row_cnt += 1
    try:
        features = sp.audio_features(df_row['url'])[0]
        df_row['danceability'] = features['danceability']
        df_row['energy'] = features['energy']
        df_row['key'] = features['key']
        df_row['loudness'] = features['loudness']
        df_row['mode'] = features['mode']
        df_row['speechiness'] = features['speechiness']
        df_row['acousticness'] = features['acousticness']
        df_row['instrumentalness'] = features['instrumentalness']
        df_row['liveness'] = features['liveness']
        df_row['valence'] = features['valence']
        df_row['tempo'] = features['tempo']
        df_row['duration_ms'] = features['duration_ms']

        try:
            track_info = sp.track(df_row['url'])
            df_row['artist_url'] = track_info['artists'][0]['external_urls']['spotify']
            df_row['track_popularity'] = track_info['popularity']

            try:
                artist_info = sp.artist(df_row['artist_url'])
                df_row['artist_genres'] = artist_info['genres']
                df_row['artist_popularity'] = artist_info['popularity']

            except Exception as e:
                print(e)
                print("artist failed")
                failed_cnt += 1
        except Exception as e:
            print(e)
            print("track failed")
            failed_cnt += 1
    except Exception as e:
        print(e)
        print("audio_feature failed")
        failed_cnt += 1

    if failed_cnt > 0 and failed_cnt % 10 == 0:
        print(f"Failed Count: {failed_cnt}")

    return df_row

In [6]:
# concatenate all features to dataframe by chunk of size about 50
def get_all_chunk_features(df_rows):
    global chunk_cnt, failed_chunk_cnt
    chunk_cnt += 1
    try:
        features_name = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'artist_url', 'track_popularity', 'artist_genres', 'artist_popularity']
        for feature in features_name:
            df_rows[feature] = np.nan

        chunk_features = sp.audio_features(list(df_rows['url']))
        for i in range(len(chunk_features)):
            df_rows.iloc[i, 10] = chunk_features[i]['danceability']
            df_rows.iloc[i, 11] = chunk_features[i]['energy']
            df_rows.iloc[i, 12] = chunk_features[i]['key']
            df_rows.iloc[i, 13] = chunk_features[i]['loudness']
            df_rows.iloc[i, 14] = chunk_features[i]['mode']
            df_rows.iloc[i, 15] = chunk_features[i]['speechiness']
            df_rows.iloc[i, 16] = chunk_features[i]['acousticness']
            df_rows.iloc[i, 17] = chunk_features[i]['instrumentalness']
            df_rows.iloc[i, 18] = chunk_features[i]['liveness']
            df_rows.iloc[i, 19] = chunk_features[i]['valence']
            df_rows.iloc[i, 20] = chunk_features[i]['tempo']
            df_rows.iloc[i, 21] = chunk_features[i]['duration_ms']

        try:
            chunk_track_info = sp.tracks(list(df_rows['url']))
            for i in range(len(chunk_track_info['tracks'])):
                df_rows.iloc[i, 22] = chunk_track_info['tracks'][i]['artists'][0]['external_urls']['spotify']
                df_rows.iloc[i, 23] = chunk_track_info['tracks'][i]['popularity']

            try:
                chunk_artist_info = sp.artists(list(df_rows['artist_url']))
                df_rows['artist_genres'] = [[] for _ in range(len(df_rows.index))]
                for i in range(len(chunk_artist_info['artists'])):
                    df_rows.at[i, 'artist_genres'] = chunk_artist_info['artists'][i]['genres']
                    df_rows.at[i, 'artist_popularity'] = chunk_artist_info['artists'][i]['popularity']

            except Exception as e:
                print(e)
                print("artists failed")
                failed_chunk_cnt += 1
        except Exception as e:
            print(e)
            print("tracks failed")
            failed_chunk_cnt += 1
    except Exception as e:
        print(e)
        print("audio_features failed")
        failed_chunk_cnt += 1

    return df_rows

In [7]:
import signal
# set timeout because sometimes a row might take too long due to the api rate limit
def handle_timeout(signum, frame):
    # This function will be called when the timeout is reached.
    raise Exception("Timeout reached")

def get_all_features_with_timeout(row):
    # Set a timeout of 10 seconds.
    signal.signal(signal.SIGALRM, handle_timeout)
    signal.alarm(10)

    try:
        result = get_all_features(row)
        signal.alarm(0)  # Cancel the timeout.
        return result
    except Exception as e:
        print(e)
        signal.alarm(0)  # Cancel the timeout.
        return row

def get_all_chunk_features_with_timeout(rows):
    # Set a timeout of 120 seconds.
    signal.signal(signal.SIGALRM, handle_timeout)
    signal.alarm(120)

    try:
        result = get_all_chunk_features(rows)
        signal.alarm(0)  # Cancel the timeout.
        return result
    except Exception as e:
        print(e)
        signal.alarm(0)  # Cancel the timeout.
        return rows

In [9]:
import numpy as np
# split dataframe of size 296597 to 100 splits
splitted_df = np.array_split(df, 100)
print(len(splitted_df))
print(splitted_df[0].shape)

100
(2966, 10)


In [10]:
last_split = 72
split_idx = [i for i in range(last_split, 100)]
# Split these dataframe and process them one by one, so if it stucks on any split, I can continue start running by that split.
for i in split_idx:
    chunk_cnt = 0
    failed_chunk_cnt = 0
    # Moreover, split every split to 60 chunks, every chunk has length about 50 because spotipy's limit
    chunks = np.array_split(splitted_df[i], 60)
    for j in range(60):
        chunks[j] = chunks[j].reset_index(drop=True)
        chunks[j] = get_all_chunk_features_with_timeout(chunks[j])
    # splitted_df[i] = splitted_df[i].progress_apply(get_all_features_with_timeout, axis=1)
    result_df = pd.concat(chunks, axis=0)
    print(f"Chunk Count: {chunk_cnt}, Failed Chunk Count: {failed_chunk_cnt}")
    result_df.to_csv(f"./outputs/final_outputs_{i}.csv", encoding='utf-8', index=False)

Chunk Count: 60, Failed Chunk Count: 0
'NoneType' object is not subscriptable
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
http status: 400, code:-1 - Unsupported URL / URI., reason: None
audio_features failed
Chunk Count: 60, Failed Chunk Count: 5
http status: 400, code:-1 - Unsupported URL / URI., reason: None
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
Chunk Count: 60, Failed Chunk Count: 3
http status: 400, code:-1 - Unsupported URL / URI., reason: None
audio_features failed
'NoneType' object is not subscriptable
audio_features failed
Chunk Count: 60, Failed Chunk Count: 2
'NoneType' object is not subscriptable
audio_features failed
Chunk Count: 60, Failed Chunk Count: 1
http status: 400, code:-1 - Unsupported URL / URI., reas

In [8]:
import os

# Set directory path to the folder containing CSV files
directory_path = "./outputs"

# Initialize an empty list to store dataframes
dfs = []

# Loop over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Read the CSV file into a dataframe
        filepath = os.path.join(directory_path, filename)
        df = pd.read_csv(filepath, header=0, encoding = "utf-8")

        # Add the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
all_dfs = pd.concat(dfs, ignore_index=True)
print(all_dfs.shape)
all_dfs.head(10)

(296597, 26)


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_url,track_popularity,artist_genres,artist_popularity
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6.0,-3.555,0.0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504.0,https://open.spotify.com/artist/7qYeIN2r4H1kBv...,67.0,"['latin hip hop', 'perreo', 'reggaeton', 'regg...",67.0
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1.0,-6.832,1.0,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808.0,https://open.spotify.com/artist/4D2d63igYEdzhz...,60.0,"['argentine hip hop', 'trap argentino']",57.0
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11.0,-8.524,0.0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339.0,https://open.spotify.com/artist/2x7PC78TmgqpEI...,0.0,"['argentine hip hop', 'bases de freestyle', 't...",73.0
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9.0,-5.421,0.0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666.0,https://open.spotify.com/artist/0GM7qgcRCORpGn...,67.0,"['latin hip hop', 'latin pop', 'pop reggaeton'...",77.0
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11.0,-3.962,0.0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213.0,https://open.spotify.com/artist/2UqRkW2wfEkZmy...,0.0,['cuarteto'],61.0
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5.0,-4.891,0.0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187.0,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,0.0,"['conscious hip hop', 'hip hop', 'rap', 'west ...",90.0
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8.0,-7.064,1.0,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560.0,https://open.spotify.com/artist/5y2Xq6xcjJb2jV...,0.0,"['neo mellow', 'neo soul', 'pop', 'pop soul', ...",78.0
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10.0,-3.194,1.0,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200.0,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,62.0,['electro house'],81.0
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3.0,-7.320,0.0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133.0,https://open.spotify.com/artist/4lxfqrEsLX6N1N...,74.0,"['mellow gold', 'rock drums', 'soft rock']",77.0
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8.0,-8.402,1.0,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182.0,https://open.spotify.com/artist/716NhGYqD1jl2w...,9.0,"['argentine hip hop', 'pop venezolano', 'trap ...",88.0


In [9]:
import numpy as np
# split dataframe of size 296597 to 10 splits
splitted_all_df = np.array_split(all_dfs, 10)
print(len(splitted_all_df))
print(splitted_all_df[0].shape)

10
(29660, 26)


In [10]:
for i in range(0, 10):
    df = splitted_all_df[i].copy()
    print(f"original all dataset shape: {df.shape}")
    print(f"row count with nan: {(df.isna().any(axis=1)).sum()}")
    row_cnt, failed_cnt = 0, 0
    df[df.isna().any(axis=1)] = df[df.isna().any(axis=1)].progress_apply(get_all_features_with_timeout, axis=1)
    print(f"Row Count: {row_cnt}, Failed Count: {failed_cnt}")
    print(f"row count with nan after processed: {(df.isna().any(axis=1)).sum()}")
    df = df.dropna()
    print(f"all dataset shape after processed: {df.shape}")
    df.to_csv(f"./outputs/all_final_outputs_{i}.csv", encoding='utf-8', index=False)

original all dataset shape: (29660, 26)
row count with nan: 1139


  0%|          | 5/1139 [00:10<1:07:10,  3.55s/it]

Timeout reached
track failed
'NoneType' object is not subscriptable
audio_feature failed


  1%|          | 7/1139 [00:20<1:20:49,  4.28s/it]

Timeout reached
track failed


  1%|          | 7/1139 [00:24<1:05:26,  3.47s/it]


KeyboardInterrupt: 

: 

### All processes is done above. Below codes are just for test

In [ ]:
df.to_csv('./final_outputs.csv', encoding='utf-8', index=False)

In [11]:
sp.audio_features(['https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF', 'https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF'])[0]

{'danceability': 0.828,
 'energy': 0.724,
 'key': 6,
 'loudness': -3.555,
 'mode': 0,
 'speechiness': 0.0715,
 'acousticness': 0.0424,
 'instrumentalness': 1.68e-06,
 'liveness': 0.468,
 'valence': 0.896,
 'tempo': 96.058,
 'type': 'audio_features',
 'id': '4vv8wcnT04GHHbUCFBAiHF',
 'uri': 'spotify:track:4vv8wcnT04GHHbUCFBAiHF',
 'track_href': 'https://api.spotify.com/v1/tracks/4vv8wcnT04GHHbUCFBAiHF',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4vv8wcnT04GHHbUCFBAiHF',
 'duration_ms': 167504,
 'time_signature': 4}

In [19]:
temp_df = df.iloc[:100, :]
temp_df.shape

(100, 10)

In [23]:
temp_df['test'] = [[] for _ in range(len(temp_df.index))]
temp_df.at[0, 'test'] = ['a', 'b', 'c']
temp_df.head()

/tmp/ipykernel_50458/4019165760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['test'] = [[] for _ in range(len(temp_df.index))]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,test
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,"[a, b, c]"
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,[]
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,[]
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,[]
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,[]


: 

In [19]:
def get_audio_features(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        features = sp.audio_features(df_row['url'])[0]
        df_row['danceability'] = features['danceability']
        df_row['energy'] = features['energy']
        df_row['key'] = features['key']
        df_row['loudness'] = features['loudness']
        df_row['mode'] = features['mode']
        df_row['speechiness'] = features['speechiness']
        df_row['acousticness'] = features['acousticness']
        df_row['instrumentalness'] = features['instrumentalness']
        df_row['liveness'] = features['liveness']
        df_row['valence'] = features['valence']
        df_row['tempo'] = features['tempo']
        df_row['duration_ms'] = features['duration_ms']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [24]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_audio_features, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:02<00:00, 38.88it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182


In [1]:
sp.tracks(['https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF', 'https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF'])['tracks'][0]['artists'][0]['external_urls']['spotify']

NameError: name 'sp' is not defined

In [25]:
def get_track_info(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        track_info = sp.track(df_row['url'])
        df_row['artist_url'] = track_info['artists'][0]['external_urls']['spotify']
        df_row['track_popularity'] = track_info['popularity']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [27]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_track_info, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:05<00:00, 17.67it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_url,track_popularity
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504,https://open.spotify.com/artist/7qYeIN2r4H1kBv...,67
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808,https://open.spotify.com/artist/4D2d63igYEdzhz...,60
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339,https://open.spotify.com/artist/2x7PC78TmgqpEI...,0
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666,https://open.spotify.com/artist/0GM7qgcRCORpGn...,67
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213,https://open.spotify.com/artist/2UqRkW2wfEkZmy...,0
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,0
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560,https://open.spotify.com/artist/5y2Xq6xcjJb2jV...,0
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,62
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133,https://open.spotify.com/artist/4lxfqrEsLX6N1N...,74
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182,https://open.spotify.com/artist/716NhGYqD1jl2w...,9


In [10]:
sp.artists(['https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav', 'https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav'])['artists'][0]['genres']

['latin hip hop', 'perreo', 'reggaeton', 'reggaeton flow', 'urbano latino']

In [7]:
chunk_track_info = sp.tracks(['https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF', 'https://open.spotify.com/track/4vv8wcnT04GHHbUCFBAiHF'])
for i in range(len(chunk_track_info['tracks'])):
    print(chunk_track_info['tracks'][i]['artists'][0]['external_urls']['spotify'])
    print(chunk_track_info['tracks'][i]['popularity'])

chunk_artist_info = sp.artists(['https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav', 'https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav'])
for i in range(len(chunk_artist_info['artists'])):
    print(chunk_artist_info['artists'][i]['genres'])
    print(chunk_artist_info['artists'][i]['popularity'])

https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav
67
https://open.spotify.com/artist/7qYeIN2r4H1kBvr0Gm9Iav
67
['latin hip hop', 'perreo', 'reggaeton', 'reggaeton flow', 'urbano latino']
67
['latin hip hop', 'perreo', 'reggaeton', 'reggaeton flow', 'urbano latino']
67


In [32]:
def get_artist_info(df_row):
    global row_cnt, success_cnt
    row_cnt += 1
    if row_cnt % 1000 == 0:
        print(f"Row Count: {row_cnt}")
    try:
        artist_info = sp.artist(df_row['artist_url'])
        df_row['artist_genres'] = artist_info['genres']
        df_row['artist_popularity'] = artist_info['popularity']
        success_cnt += 1
        if success_cnt % 1000 == 0:
            print(f"Success Count: {success_cnt}")
    except:
        pass
    
    return df_row

In [33]:
row_cnt = 0
success_cnt = 0
temp_df = temp_df.progress_apply(get_artist_info, axis=1)
temp_df.head(10)

100%|██████████| 100/100 [00:03<00:00, 32.58it/s]


,title,rank,date,artist,url,region,chart,trend,streams,min_rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,artist_url,track_popularity,artist_genres,artist_popularity
0,23 (With Ape Drums),157,10/22/2021,Randy,https://open.spotify.com/track/4vv8wcnT04GHHbU...,Argentina,top200,NEW_ENTRY,37919,76,0.828,0.724,6,-3.555,0,0.0715,0.0424,0.000002,0.4680,0.896,96.058,167504,https://open.spotify.com/artist/7qYeIN2r4H1kBv...,67,"[latin hip hop, perreo, reggaeton, reggaeton f...",67
1,24K,116,9/9/2019,"Frijo, Santoz, Duki",https://open.spotify.com/track/1sbxPVKNi6OLUdT...,Argentina,top200,NEW_ENTRY,31528,108,0.810,0.833,1,-6.832,1,0.1250,0.2530,0.005760,0.2160,0.907,138.955,123808,https://open.spotify.com/artist/4D2d63igYEdzhz...,60,"[argentine hip hop, trap argentino]",57
2,4:00 AM,193,2/23/2018,Trueno,https://open.spotify.com/track/2rN6ZDtVTPflMgf...,Argentina,top200,NEW_ENTRY,16786,193,0.804,0.590,11,-8.524,0,0.2640,0.1700,0.000000,0.1060,0.453,123.947,247339,https://open.spotify.com/artist/2x7PC78TmgqpEI...,0,"[argentine hip hop, bases de freestyle, trap a...",74
3,Agua (with J Balvin),145,3/5/2021,Tainy,https://open.spotify.com/track/7CtbxAGq2P0jtYh...,Argentina,top200,MOVE_UP,33015,124,0.778,0.651,9,-5.421,0,0.2660,0.1670,0.000000,0.1770,0.569,100.094,159666,https://open.spotify.com/artist/0GM7qgcRCORpGn...,67,"[latin hip hop, pop reggaeton, reggaeton, regg...",77
4,Ahora mirame,199,7/1/2017,Ulises Bueno,https://open.spotify.com/track/780CrNUSw5RrRWH...,Argentina,top200,NEW_ENTRY,13323,153,0.735,0.917,11,-3.962,0,0.0389,0.0673,0.000000,0.0453,0.811,149.961,274213,https://open.spotify.com/artist/2UqRkW2wfEkZmy...,0,[cuarteto],61
5,All The Stars (with SZA),190,2/9/2018,Kendrick Lamar,https://open.spotify.com/track/2Imq77E6CEJR5TP...,Argentina,top200,NEW_ENTRY,16524,124,0.532,0.642,5,-4.891,0,0.1110,0.0548,0.000246,0.0991,0.536,193.016,232187,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,0,"[conscious hip hop, hip hop, rap, west coast rap]",90
6,All of Me,193,2/6/2017,John Legend,https://open.spotify.com/track/2q6tgiA9tpb6DMg...,Argentina,top200,NEW_ENTRY,11296,89,0.422,0.264,8,-7.064,1,0.0322,0.9220,0.000000,0.1320,0.331,119.930,269560,https://open.spotify.com/artist/5y2Xq6xcjJb2jV...,0,"[neo mellow, neo soul, pop, pop soul, r&b, urb...",78
7,Alone,190,1/30/2017,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Argentina,top200,NEW_ENTRY,12091,183,0.676,0.929,10,-3.194,1,0.0458,0.1860,0.000405,0.1210,0.157,97.019,161200,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,62,[electro house],81
8,Another Day in Paradise - 2016 Remaster,188,3/21/2018,Phil Collins,https://open.spotify.com/track/1NCuYqMc8hKMb4c...,Argentina,top200,NEW_ENTRY,15307,188,0.780,0.565,3,-7.320,0,0.0306,0.7360,0.002690,0.0630,0.365,101.967,322133,https://open.spotify.com/artist/4lxfqrEsLX6N1N...,74,"[mellow gold, rock drums, soft rock]",77
9,"Asan: Bzrp Music Sessions, Vol. 35",69,11/12/2020,"Bizarrap, Asan",https://open.spotify.com/track/5ZoYsrmeVPyJI4r...,Argentina,top200,NEW_ENTRY,54640,69,0.680,0.576,8,-8.402,1,0.0866,0.2050,0.000000,0.1090,0.507,164.978,154182,https://open.spotify.com/artist/716NhGYqD1jl2w...,9,"[argentine hip hop, pop venezolano, trap argen...",88
